# Coursework 2: Image segmentation

In this coursework you will develop and train a convolutional neural network for brain tumour image segmentation. Please read both the text and the code in this notebook to get an idea what you are expected to implement. Pay attention to the missing code blocks that look like this:

```
### Insert your code ###
...
### End of your code ###
```

## What to do?

* Complete and run the code using `jupyter-lab` or `jupyter-notebook` to get the results.

* Export (File | Save and Export Notebook As...) the notebook as a PDF file, which contains your code, results and answers, and upload the PDF file onto [Scientia](https://scientia.doc.ic.ac.uk).

* Instead of clicking the Export button, you can also run the following command instead: `jupyter nbconvert coursework.ipynb --to pdf`

* If Jupyter complains about some problems in exporting, it is likely that pandoc (https://pandoc.org/installing.html) or latex is not installed, or their paths have not been included. You can install the relevant libraries and retry.

* If Jupyter-lab does not work for you at the end, you can use Google Colab to write the code and export the PDF file.

## Dependencies

You need to install Jupyter-Lab (https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html) and other libraries used in this coursework, such as by running the command:
`pip3 install [package_name]`

## GPU resource

The coursework is developed to be able to run on CPU, as all images have been pre-processed to be 2D and of a smaller size, compared to original 3D volumes.

However, to save training time, you may want to use GPU. In that case, you can run this notebook on Google Colab. On Google Colab, go to the menu, Runtime - Change runtime type, and select **GPU** as the hardware acceleartor. At the end, please still export everything and submit as a PDF file on Scientia.


In [1]:
# Import libraries
# These libraries should be sufficient for this tutorial.
# However, if any other library is needed, please install by yourself.
import tarfile
import imageio.v3
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
import time
import os
import random
import matplotlib.pyplot as plt
from matplotlib import colors

ModuleNotFoundError: No module named 'torch._C'

## 1. Download and visualise the imaging dataset.

The dataset is curated from the brain imaging dataset in [Medical Decathlon Challenge](http://medicaldecathlon.com/). To save the storage and reduce the computational cost for this tutorial, we extract 2D image slices from T1-Gd contrast enhanced 3D brain volumes and downsample the images.

The dataset consists of a training set and a test set. Each image is of dimension 120 x 120, with a corresponding label map of the same dimension. There are four number of classes in the label map:

- 0: background
- 1: edema
- 2: non-enhancing tumour
- 3: enhancing tumour

In [ ]:
# Download the dataset
!wget https://www.dropbox.com/s/zmytk2yu284af6t/Task01_BrainTumour_2D.tar.gz

# Unzip the '.tar.gz' file to the current directory
datafile = tarfile.open('Task01_BrainTumour_2D.tar.gz')
datafile.extractall()
datafile.close()

Suggested colour map for brain MR image:
```
cmap = 'gray'
```

Suggested colour map for segmentation map:
```
cmap = colors.ListedColormap(['black', 'green', 'blue', 'red'])
```

In [ ]:
from pathlib import Path

IMAGES_DIR = Path('./Task01_BrainTumour_2D')

TRAINING_IMAGES_DIR = IMAGES_DIR / 'training_images/'
TRAINING_LABELS_DIR = IMAGES_DIR / 'training_labels/'

def extract_images_from(dir_path):
    images = {}
    for file in dir_path.iterdir():
        if not file.is_file():
            continue
        images[file.name] = imageio.v3.imread(file)
    return images
        
training_images_dict = extract_images_from(TRAINING_IMAGES_DIR)
training_labels_dict = extract_images_from(TRAINING_LABELS_DIR)

def zip_images_and_labels(image_dict, label_dict):
    pairs = []
    for file_name in image_dict.keys():
        pairs.append((image_dict[file_name], label_dict[file_name]))
    return pairs

images_with_labels = zip_images_and_labels(training_images_dict, training_labels_dict)
    
NUMBER_OF_IMAGES_TO_VISUALISE = 4

images_with_labels = random.sample(images_with_labels, NUMBER_OF_IMAGES_TO_VISUALISE)

# For visualising the labels, the following color map was used
# 0 -> 'black' : background
# 1 -> 'green' : edema
# 2 -> 'blue'  : non-enhancing tumour
# 3 -> 'red'   : enhancing tumour
def visualise(images_with_labels):
    figure = plt.figure(figsize=(8, 8))
    columns = 2 # In each row we visualise the image and its labels.
    rows = len(images_with_labels)
    for i, (image, labels) in enumerate(images_with_labels):
        figure.add_subplot(rows, columns, 2*i + 1)
        plt.imshow(image, cmap='gray')
        plt.gcf().set_size_inches(8, 8)
        figure.add_subplot(rows, columns, 2*i + 2)
        plt.imshow(labels, cmap=colors.ListedColormap(['black', 'green', 'blue', 'red']))
        plt.gcf().set_size_inches(8, 8)
        
    plt.show()
        
visualise(images_with_labels)

## 2. Implement a dataset class.

It can read the imaging dataset and get items, pairs of images and label maps, as training batches.

In [ ]:
def normalise_intensity(image, thres_roi=1.0):
    """ Normalise the image intensity by the mean and standard deviation """
    # ROI defines the image foreground
    val_l = np.percentile(image, thres_roi)
    roi = (image >= val_l)
    mu, sigma = np.mean(image[roi]), np.std(image[roi])
    eps = 1e-6
    image2 = (image - mu) / (sigma + eps)
    return image2


class BrainImageSet(Dataset):
    """ Brain image set """
    def __init__(self, image_path, label_path='', deploy=False):
        self.image_path = Path(image_path)
        self.label_path = Path(label_path)
        self.deploy = deploy
        self.images = []
        self.labels = []

        images_dict = extract_images_from(self.image_path)
        
        # Swapped the code for reading in images because the imageio.v3 library
        # was complaining about it.
        if not self.deploy:
            labels_dict = extract_images_from(self.label_path)
            images_with_labels = zip_images_and_labels(images_dict, labels_dict)
            images, labels = list(zip(*images_with_labels))
            # Needed to reshape the array here so that pytorch deals with it correctly.
            self.images = np.array([np.reshape(image, (1, 120, 120)) for image in images])
            self.labels = np.array([np.reshape(label, (1, 120, 120)) for label in labels])
        else:
            self.images = images.values()
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get an image and perform intensity normalisation
        # Dimension: XY
        image = normalise_intensity(self.images[idx])

        # Get its label map
        # Dimension: XY
        label = self.labels[idx]
        return image, label

    def get_random_batch(self, batch_size):
        # Get a batch of paired images and label maps
        # Dimension of images: NCXY
        # Dimension of labels: NXY
        images, labels = [], []

        random_indices = random.sample(range(len(self)), batch_size)
        
        for idx in random_indices:
            image, label = self[idx]
            images.append(image)
            labels.append(label)
        return np.array(images), np.array(labels)

## 3. Build a U-net architecture.

You will implement a U-net architecture. If you are not familiar with U-net, please read this paper:

[1] Olaf Ronneberger et al. [U-Net: Convolutional networks for biomedical image segmentation](https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28). MICCAI, 2015.

For the first convolutional layer, you can start with 16 filters. We have implemented the encoder path. Please complete the decoder path.

In [ ]:
""" U-net """
class UNet(nn.Module):
    def __init__(self, input_channel=1, output_channel=1, num_filter=16):
        super(UNet, self).__init__()

        # BatchNorm: by default during training this layer keeps running estimates
        # of its computed mean and variance, which are then used for normalization
        # during evaluation.

        # Encoder path
        n = num_filter  # 16
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channel, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n *= 2  # 32
        self.conv2 = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n *= 2  # 64
        self.conv3 = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n *= 2  # 128
        self.conv4 = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )
        
        # The first step in each one of conv2 conv3 conv4 above is the max pooling
        # for the decode step we need to perform up-conv 2x2 and then the rest
        # of the structure remains the same (BatchNorm2d -> ReLU -> Conv2d -> BatchNorm2d -> ReLU)
        
        # Each of the decode steps also needs to take the skipped input.
        # In the paper it was stated that they used an up-convolution using a 
        # learned kernel. It means that we need to use ConvTranspose2d to have
        # a trainable weight on that layer (rather than using e.g. a simple upsampling).
        
        # In the following we need to split the layers as the U-Net takes the skipped
        # outputs from the previous convolution layers and concatenates them with 
        # the intermediate steps. Therefore we need to perform upconvolution and the 
        # regular conv 3x3 ReLU steps separately.
        
        # Decoder path
        # We decrease the number of output channels.
        n = int(n / 2) # 64
        self.upconv1 = nn.Sequential(
            nn.ConvTranspose2d(n * 2, n, kernel_size=2, stride=2),
            nn.BatchNorm2d(n),
            nn.ReLU(),
        )
        # This one takes in the concatenated input therefore it will still have n*2 input channels.
        self.conv5 = nn.Sequential(
            nn.Conv2d(n*2, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )
        
        n = int(n / 2) # 32
        self.upconv2 = nn.Sequential(
            nn.ConvTranspose2d(n * 2, n, kernel_size=2, stride=2),
            nn.BatchNorm2d(n),
            nn.ReLU(),
        )
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(n*2, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n = int(n / 2) # 16
        self.upconv3 = nn.Sequential(
            nn.ConvTranspose2d(n * 2, n, kernel_size=2, stride=2),
            nn.BatchNorm2d(n),
            nn.ReLU(),
        )
        
        # For the output layer we need conv 1x1 as described in the paper
        NUMBER_OF_CLASSES = 4 # Need a separate output for each class.
        self.conv7 = nn.Sequential(
            nn.Conv2d(n*2, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, NUMBER_OF_CLASSES, kernel_size=1)
        )
        
        

    def forward(self, x):
        # Use the convolutional operators defined above to build the U-net
        # The encoder part is already done for you.
        # You need to complete the decoder part.
        # Encoder
        x = self.conv1(x)
        conv1_skip = x

        x = self.conv2(x)
        conv2_skip = x

        x = self.conv3(x)
        conv3_skip = x

        x = self.conv4(x)
        
        # Decoder
        
        # First we perform up-convolution and then concatenate with the skipped values
        # Question: does the order of concatenation matter?
        # After concatenation we perform the regular conv 3x3 ReLU.
        
        x = self.upconv1(x)
        x = torch.cat((conv3_skip, x), 1)
        x = self.conv5(x)
        
        x = self.upconv2(x)
        x = torch.cat((conv2_skip, x), 1)
        x = self.conv6(x)
        
        x = self.upconv3(x)
        x = torch.cat((conv1_skip, x), 1)
        x = self.conv7(x)
        
        return x

## 4. Train the segmentation model.

In [ ]:
# CUDA device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))

# Build the model
num_class = 4
model = UNet(input_channel=1, output_channel=num_class, num_filter=16)
model = model.to(device)
params = list(model.parameters())

model_dir = 'saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Optimizer
optimizer = optim.Adam(params, lr=1e-3)

# Segmentation loss
criterion = nn.CrossEntropyLoss()

# Datasets
train_set = BrainImageSet('./Task01_BrainTumour_2D/training_images/', './Task01_BrainTumour_2D/training_labels/')
test_set = BrainImageSet('./Task01_BrainTumour_2D/test_images/', './Task01_BrainTumour_2D/test_labels/')

# Train the model
# Note: when you debug the model, you may reduce the number of iterations or batch size to save time.
num_iter = 10000
train_batch_size = 16
eval_batch_size = 16
start = time.time()

for it in range(1, 1 + num_iter):
    # Set the modules in training mode, which will have effects on certain modules, e.g. dropout or batchnorm.
    start_iter = time.time()
    model.train()

    # Get a batch of images and labels
    images, labels = train_set.get_random_batch(train_batch_size)
    images, labels = torch.from_numpy(images), torch.from_numpy(labels)
    images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)

    # Perform optimisation and print out the training loss
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(torch.squeeze(output), torch.squeeze(labels))
    loss.backward()
    optimizer.step()
    # As instructed on edStem, the printing while training is disabled.
    #print("Iteration: {}, Training loss: {}".format(it, loss.item()))

    # Evaluate
    if it % 100 == 0:
        model.eval()
        # Disabling gradient calculation during reference to reduce memory consumption
        with torch.no_grad():
            # Evaluate on a batch of test images and print out the test loss
            images, labels = test_set.get_random_batch(train_batch_size)
            images, labels = torch.from_numpy(images), torch.from_numpy(labels)
            images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
            output = model(images)
            loss = criterion(torch.squeeze(output), torch.squeeze(labels))
            print("Iteration: {}, Test loss: {}".format(it, loss.item()))
 

    # Save the model
    if it % 1000 == 0:
        torch.save(model.state_dict(), os.path.join(model_dir, 'model_{0}.pt'.format(it)))
print('Training took {:.3f}s in total.'.format(time.time() - start))

## 5. Deploy the trained model to a random set of 4 test images and visualise the automated segmentation.

You can show the images as a 4 x 3 panel. Each row shows one example, with the 3 columns being the test image, automated segmentation and ground truth segmentation.

In [ ]:
images, labels = test_set.get_random_batch(4)
images, labels = torch.from_numpy(images), torch.from_numpy(labels)
images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)

model.load_state_dict(torch.load(os.path.join(model_dir, 'model_10000.pt')))

output = model(images)

def visualise(images, labels, output):
    with torch.no_grad():
        images = torch.squeeze(images).detach()
        labels = torch.squeeze(labels).detach()
        # Need to produce a single image from output
        outputs = torch.squeeze(output).detach()
        merged_outputs = []
        for output in outputs:
            # apply thresholding so that the black one doesn't always win
            merged_outputs.append(np.argmax(output, axis = 0))
            
        figure = plt.figure(figsize=(8, 8))
        columns = 3 
        rows = 4
        plt.figtext(0.5, 0.05, '3 columns represent: test image, automated segmentation and ground truth segmentation in order.', horizontalalignment='center',
             verticalalignment='top')
        for i, (image, label, output) in enumerate(zip(images, labels, merged_outputs)):
            figure.add_subplot(rows, columns, 3*i + 1)
            plt.imshow(image, cmap='gray')
            plt.gcf().set_size_inches(8, 8)
            figure.add_subplot(rows, columns, 3*i + 3)
            plt.imshow(output, cmap=colors.ListedColormap(['black', 'green', 'blue', 'red']))
            plt.gcf().set_size_inches(8, 8)
            figure.add_subplot(rows, columns, 3*i + 2)
            plt.imshow(label, cmap=colors.ListedColormap(['black', 'green', 'blue', 'red']))
            plt.gcf().set_size_inches(8, 8)
            

        plt.show()
        
        
loss_after_iterations = []
for i in range(1, 11):
    model.load_state_dict(torch.load(os.path.join(model_dir, 'model_{}000.pt'.format(i))))
    print("Model produced after {}000 iterations:".format(i))
    loss = criterion(torch.squeeze(output), torch.squeeze(labels))
    print("Test loss: {}".format(loss.item()))
    loss_after_iterations.append(loss.item())

    output = model(images)
    visualise(images, labels, output)
    
x = np.array(["{}000".format(i) for i in range(1, 11)])
y = np.array(loss_after_iterations)

print("Test error for each 1000 of iterations:")
plt.plot(x, y)
             

## 6. Discussion. Does your trained model work well? How would you improve this model so it can be deployed to the real clinic?

As it could be observed in the output image segmentation above, the model gives automated segmentation images which visually resemble the ground truth values.
One thing I've noticed is that the test error sometimes doesn't fully resemble the visual correspondence of the images. For example in the outputs generated for the 
model after 8000 iterations have the lowest test error out of all models (see the graph above) however for the first picture the whole tumour is marked with red and it should be blue if we examine the ground truth.
We can contrast that with the output after 10000 iterations, where the tumour is correctly labeled in blue, however the overall error is larger. 

This inconsistency could be attributed to the fact that we compute the loss over the whole batch of 4 images and it could be that the model performed worse on the
other images in that case. 

Another thing to notice is the overfitting present in the model. One could observe by looking at the training output above that the training error keept decreasing as I was training the model, 
however the test error seems not to be affected after some point. That could also be seen in the plot above, where the performance after 9000 iterations is worse than after 8. 
It is important to note however, that all of those scores were computed only for small batches of images and so in order to accurately compare the performance of the models we would have to do it for the whole test dataset. 

Overall I think the trained models perform well provided that they are trained for at least 4000 iterations. Both the test scores computed using the cross-entropy loss and the visual resemblance are quite good.

In order to improve the model so that it can be deployed in the real clinic I would probably consider performing some more advanced hyperparameter tuning to improve the performance. One could adapt the learning rate for the Adam optimiser used in the network for example. I could also try to tweak the size of a training batch. Before using the model in a production environment, I would probably perform some kind of exhaustive grid search to find the best hyperparameters to pick when training the model.

I would also consider using a larger dataset to train the model. If the data is not available, we could introduce some transformations to the images to artificially increase the number of training data (e.g. some deformations or smooth stretching). 

If the model were to be used in an actual medical setting, it needs to be very robust as human life is at stake. Especially false negatives could be very dangerous. The model should be able to reliably detect presence of some kind of 
tumour, and its actual type has lower priority. 